In [0]:
dbutils.widgets.text('prm-processName','')
#print(dbutils.widgets.get('prm-processName'))
processName=dbutils.widgets.get('prm-processName')

nextSourceFileDateSql=f"select nvl(max(processed_file_table_date)+1, CAST('2023-01-01' AS DATE)) as next_source_file_date from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs` where process_name='{processName}' and process_status='completed'"

#print(nextSourceFileDateSql)

nextSourceFileDateDF=spark.sql(nextSourceFileDateSql)

nextSourceFileDate=nextSourceFileDateDF.select('next_source_file_date').collect()[0]['next_source_file_date']

#print(nextSourceFileDate)

In [0]:
from datetime import datetime
#dbutils.widgets.text('prm-dailyPricingSourceFileDate','')
#print(datetime.strptime(dbutils.widgets.get('prm-dailyPricingSourceFileDate'),'%Y-%m-%d').strftime('%d%m%Y'))

In [0]:
#sourceFileURL='https://retailpricing.blob.core.windows.net/labs/lab1/PW_MW_DR_01012023.csv'
#bronzeLayerCSVFilePath='abfss://working-labs-dev@dbrkcrse20251storagedev.dfs.core.windows.net/bronze-dev/daily-pricing/csv'

In [0]:
dailyPricingSourceBaseURL='https://retailpricing.blob.core.windows.net/'
dailyPricingSourceFolder='daily-pricing/'
#dailyPricingSourceFileDate=datetime.strptime(dbutils.widgets.get('prm-dailyPricingSourceFileDate'),'%Y-%m-%d').strftime('%d%m%Y')
dailyPricingSourceFileDate=datetime.strftime(nextSourceFileDate,'%d%m%Y')
dailyPricingSourceFile=f'PW_MW_DR_{dailyPricingSourceFileDate}.csv'

dailyPricingSinkLayer='bronze-dev'
dailyPricingSinkStorageAccount='dbrkcrse20251storagedev'
dailyPricingSinkFolder='daily-pricing'

#print(dailyPricingSourceFile)

In [0]:
import pandas as pds

In [0]:
dailyPricingSourceURL=dailyPricingSourceBaseURL+dailyPricingSourceFolder+dailyPricingSourceFile
#print(dailyPricingSourceURL)


In [0]:
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceURL)

In [0]:
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)
#display(dailyPricingSparkDF)

In [0]:
from pyspark.sql import functions as func

dailyPricingSinkFolderPath=f'abfs://{dailyPricingSinkLayer}@{dailyPricingSinkStorageAccount}.dfs.core.windows.net/{dailyPricingSinkFolder}'
#print(dailyPricingSinkFolderPath)

In [0]:
dailyPricingSparkDF.withColumn("source_file_load_date",func.current_timestamp()).write.mode('append').option('header','true').csv(dailyPricingSinkFolderPath)


In [0]:
processStatus='completed'

processInsertSQL=f"insert into `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs` (process_name,processed_file_table_date,process_status) values ('{processName}','{nextSourceFileDate}','{processStatus}')"

spark.sql(processInsertSQL)



In [0]:
%sql
select * from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs` order by processed_file_table_date

In [0]:
%sql
-- select max(processed_file_table_date)+1 as next_source_file_date from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`
-- where process_name='dailyPricingSourceIngest' and process_status='completed'

In [0]:
# dbutils.fs.ls('abfs://bronze-dev@dbrkcrse20251storagedev.dfs.core.windows.net/daily-pricing')
# dbutils.fs.rm('abfs://bronze-dev@dbrkcrse20251storagedev.dfs.core.windows.net/daily-pricing', recurse=True)

In [0]:
%sql
-- truncate table `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`;
-- select * from `pricing-analytics`.`processrunlogs`.`deltalakehouse_process_runs`;